In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_815622/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 16:03:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/16 16:03:07 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Listens table
listens_data = {
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5],
    'song_id': [10, 11, 12, 10, 11, 12, 10, 11, 12, 10, 11, 13, 10, 11, 12],
    'day': ['2021-03-15']*12 + ['2021-03-16']*3
}

# Sample data for the Friendship table
friendship_data = {
    'user1_id': [1],
    'user2_id': [2]
}

# Convert data to DataFrames
listens_df = pd.DataFrame(listens_data)
friendship_df = pd.DataFrame(friendship_data)

# Display the DataFrames
print("Listens table:")
print(listens_df)
print("\nFriendship table:")
print(friendship_df)



df_person = spark.createDataFrame(listens_df)
df_person.createOrReplaceTempView("Listens")

df_person = spark.createDataFrame(friendship_df)
df_person.createOrReplaceTempView("Friendship")



Listens table:
    user_id  song_id         day
0         1       10  2021-03-15
1         1       11  2021-03-15
2         1       12  2021-03-15
3         2       10  2021-03-15
4         2       11  2021-03-15
5         2       12  2021-03-15
6         3       10  2021-03-15
7         3       11  2021-03-15
8         3       12  2021-03-15
9         4       10  2021-03-15
10        4       11  2021-03-15
11        4       13  2021-03-15
12        5       10  2021-03-16
13        5       11  2021-03-16
14        5       12  2021-03-16

Friendship table:
   user1_id  user2_id
0         1         2


In [36]:
query = """
    select
        user_id,
        same_person as recommended_id
    from (
        select
            distinct 
            user_id,
            same_person,
            count_same
        from (
            select
                *,
                count(*) over(partition by day, user_id, same_person) as count_same
            from (
                select
                    distinct
                    l.day,
                    l.user_id,
                    l1.user_id as same_person,
                    l.song_id
                from
                    Listens l 
                    left join Listens l1 
                        on l.day = l1.day 
                        and l.song_id = l1.song_id 
                        and l.user_id != l1.user_id
                order by day, user_id, same_person, song_id
            )
        )
        where 
            same_person is not null 
            and count_same >= 3
            and (least(user_id, same_person), greatest(user_id, same_person)) not in (
                select user1_id, user2_id from Friendship
            )
    )
"""

In [37]:
spark.sql(query).show(50)

+-------+--------------+
|user_id|recommended_id|
+-------+--------------+
|      3|             2|
|      1|             3|
|      2|             3|
|      3|             1|
+-------+--------------+

